In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
# @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
import Mycelia

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
modified_fasta_files = sort(filter(x -> occursin(r"\.fna$", x) && occursin(".vcf", x) && occursin("normalized", x), readdir(genome_dir, join=true)), by=x->filesize(x))

In [ ]:
# fasta_file = first(fasta_files)
# coverage = 10
# # coverage = 100
# # coverage = 1000

In [ ]:
threads = 1
memory = 2
for fasta_file in modified_fasta_files
    for coverage in [10, 100, 1000]
        outprefix = "$(fasta_file).art.$(coverage)x."
        forward = outprefix * "1.fq.gz"
        reverse = outprefix * "2.fq.gz"
        if isfile(forward) && isfile(reverse) && (filesize(forward) > 0) && (filesize(reverse) > 0)
            @info "$(forward) and $(reverse) found, skipping..."
            continue
        end        
        cmd = 
        """
        $(Mycelia.MAMBA) run --live-stream -n art art_illumina --noALN --seqSys HS25 --paired --len 150 --mflen 500 --sdev 10 --in $(fasta_file) --fcov $(coverage) --out $(fasta_file).art.$(coverage)x. \\
        && gzip $(fasta_file).art.$(coverage)x.1.fq \\
        && gzip $(fasta_file).art.$(coverage)x.2.fq
        """
        Mycelia.nersc_sbatch(
            job_name = "$(basename(outprefix))",
            mail_user = "cameron.prybol@gmail.com",
            logdir = mkpath("$(homedir())/workspace/slurmlogs"),
            qos = "shared",
            mem_gb = memory,
            cpus_per_task= threads,
            cmd = cmd)
    end
end